# Load data

In [ ]:
import pandas as pd
df = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
df.head()

In [ ]:
cat_cols = [x for x in df.columns if x.startswith('cat')]
cat_cols

In [ ]:
for cat in cat_cols:
    df[cat] = df[cat].astype("category")

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size = 0.5)

# Training a simple lightgbm model

In [ ]:
from lightgbm.sklearn import LGBMRegressor
bt = LGBMRegressor(n_jobs=1)
x_train = train.drop(columns=['target','id'])
y_train = train['target']

In [ ]:
bt.fit(x_train, y_train)

In [ ]:
x_test = test.drop(columns=['target','id'])
y_test = test['target']

In [ ]:
pred = bt.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(y_test, pred))

## Plotting the model's response

In [ ]:
from plotnine import ggplot, geom_point, aes
from plotnine.geoms import geom_histogram, geom_bar, geom_boxplot, geom_bin2d, geom_density_2d, geom_segment
from plotnine.labels import ggtitle

In [ ]:
df_plot = pd.DataFrame({'pred':pred, 'y':y_test})

In [ ]:
ggplot(df_plot, aes(x='y_test', y='pred'))+geom_bin2d(bins = 100)+geom_segment(aes(x = 2, y = 2, xend = 10, yend = 10))

In [ ]:
df_plot_dist = pd.melt(df_plot)
from plotnine.facets import facet_grid
ggplot(df_plot_dist, aes(x='value', ))+geom_histogram(bins=100)+facet_grid('variable ~ .')

## Conclusion
The LightGBM model does a very poor job of capturing the bimodel distribution of the output. Is there any way we can do better?